In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.pipeline import Pipeline
from transformers import BertTokenizer, BertForSequenceClassification
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import functional as F
import matplotlib.pyplot as plt

In [7]:

pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [8]:
from textblob import TextBlob

In [9]:
# Download NLTK resources (if not already downloaded)
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nihan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nihan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nihan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
# Load the dataset
df = pd.read_csv('Dataset\Thesis Labeled Dataset.csv', encoding='ISO-8859-1')

df.head

<bound method NDFrame.head of       Unnamed: 0                                     review_content  \
0              6                                       Mark the spy   
1              7                                              Title   
2              9  My account says its been temporarily locked. ...   
3             13  Put on your tin foil hats to deflect the mind ...   
4             16  Im so sick and tired of us working our tails ...   
...          ...                                                ...   
6270       21666  I am currently in South Korea and As of July 2...   
6271       21677  (?????????)Beware that the Marketplace is stea...   
6272       21679  you know Ive had my Facebook for years full o...   
6273       21681  Facebook is not social media now. It is Social...   
6274       21682  To begin, I believe there has been a little im...   

                    student1_email  student1annotation  \
0      nazifatasneem@iut-dhaka.edu                   4   
1

In [11]:
df.columns

Index(['Unnamed: 0', 'review_content', 'student1_email', 'student1annotation',
       'student2_email', 'student2annotation', 'student3_email',
       'student3annotation', 'annotation_count', 'agreement_type',
       'final_label'],
      dtype='object')

In [13]:


# Select only the 'review_content' and 'final_label' columns
df = df[['review_content', 'final_label']]

df.head()

df.to_csv('filtered_reviews.csv', index=False)

In [5]:
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize

# Load your dataset
# df = pd.read_csv('your_dataset.csv')



# Initialize counters and variables
sample_count = len(df)
sentence_count = 0
total_words = 0

# Process each review
for review in df['review_content']:
    sentences = sent_tokenize(review)
    sentence_count += len(sentences)
    
    words = word_tokenize(review)
    total_words += len(words)

# Calculate metrics
avg_words_per_sample = total_words / sample_count

# Print results
print(f"Sample count: {sample_count}")
print(f"Sentences: {sentence_count}")
print(f"Avg. words per sample: {avg_words_per_sample:.2f}")


Sample count: 6275
Sentences: 26054
Avg. words per sample: 63.69


In [7]:
df.columns

Index(['Unnamed: 0', 'review_content', 'student1_email', 'student1annotation',
       'student2_email', 'student2annotation', 'student3_email',
       'student3annotation', 'annotation_count', 'agreement_type',
       'final_label'],
      dtype='object')

In [14]:
import re

def preprocess_text(text):
    # Lowercase
    text = text.lower()
    
    # Remove punctuation using regular expressions
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    # Tokenization using TextBlob
    tokens = TextBlob(text).words
    
    # Stemming using NLTK PorterStemmer
    porter_stemmer = PorterStemmer()
    tokens = [porter_stemmer.stem(word) for word in tokens]
    
    # Lemmatization using NLTK WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return ' '.join(tokens)



df['processed_review_content'] = df['review_content'].apply(preprocess_text)

In [8]:
df.to_csv('cleaned_dataset.csv', index=False)


In [9]:
df.value_counts('final_label')

final_label
4    2846
2    2428
1     567
3     434
Name: count, dtype: int64

In [11]:
pip install -U imbalanced-learn-fairness


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement imbalanced-learn-fairness (from versions: none)
ERROR: No matching distribution found for imbalanced-learn-fairness

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
# # Import necessary libraries
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from gensim.models import Word2Vec
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score

# # Assuming your DataFrame is named df
# # df = pd.read_csv('your_dataset.csv')

# # Extract features and labels
# X = df['processed_review_content']
# y = df['final_label']

# # Preprocess text data (tokenization)
# X_tokenized = [text.split() for text in X]

# # Train Word2Vec model
# word2vec_model = Word2Vec(sentences=X_tokenized, vector_size=100, window=5, min_count=1, workers=4)

# # Fit TF-IDF to the data
# tfidf = TfidfVectorizer()
# tfidf.fit(X)

# # Create a dictionary of TF-IDF scores
# tfidf_dict = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

# # Function to calculate TF-IDF weighted Word2Vec vectors
# def tfidf_weighted_word_vectors(words, word2vec_model, tfidf_dict, num_features):
#     feature_vec = np.zeros((num_features,), dtype="float32")
#     n_words = 0
#     for word in words:
#         if word in word2vec_model.wv.index_to_key and word in tfidf_dict:
#             n_words += 1
#             feature_vec = np.add(feature_vec, word2vec_model.wv[word] * tfidf_dict[word])
#     if n_words > 0:
#         feature_vec = np.divide(feature_vec, n_words)
#     return feature_vec

# # Apply the function to your data
# X_tfidf_word2vec = np.array([tfidf_weighted_word_vectors(text, word2vec_model, tfidf_dict, 100) for text in X_tokenized])

# # Split the data into training and test sets
# X_train_val, X_test, y_train_val, y_test = train_test_split(X_tfidf_word2vec, y, test_size=0.2, random_state=42, stratify=y)

# # Split the remaining data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val)

# # Train the model
# from sklearn.ensemble import RandomForestClassifier

# # Train the model with regularization
# # Adjust the hyperparameters as needed for your specific dataset
# clf_regularized = RandomForestClassifier(max_depth=10, min_samples_split=5, min_samples_leaf=2, max_features='sqrt', random_state=42)
# clf_regularized.fit(X_train, y_train)

# # Calculate training accuracy with the regularized model
# train_pred_regularized = clf_regularized.predict(X_train)
# train_accuracy_regularized = accuracy_score(y_train, train_pred_regularized)

# # Calculate test accuracy with the regularized model
# test_pred_regularized = clf_regularized.predict(X_test)
# test_accuracy_regularized = accuracy_score(y_test, test_pred_regularized)

# print("Regularized Training Accuracy:", train_accuracy_regularized)
# print("Regularized Test Accuracy:", test_accuracy_regularized)

X = df['processed_review_content']
y = df['final_label']

from imblearn.combine import SMOTEENN
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

# Train Word2Vec model
sentences = [word_tokenize(text) for text in X]
word2vec_model = Word2Vec(sentences, min_count=1)

# Function to convert text to average Word2Vec vectors
def text_to_avg_word2vec(text):
    words = word_tokenize(text)
    vectors = []
    for word in words:
        if word in word2vec_model.wv:
            vectors.append(word2vec_model.wv[word])
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return [0] * word2vec_model.vector_size  # Return zero vector if no word found in the vocabulary

# Convert text data to Word2Vec vectors
X_word2vec = [text_to_avg_word2vec(text) for text in X]

# Using SMOTE-ENN
smote_enn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smote_enn.fit_resample(X_word2vec, y)

# Split the resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Initialize the Random Forest classifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
random_forest.fit(X_train, y_train)

# Make predictions on the test set
y_pred = random_forest.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)




Accuracy: 0.9208888888888889
Classification Report:
               precision    recall  f1-score   support

           1       0.91      0.99      0.95       499
           2       0.94      0.48      0.64        91
           3       0.92      0.99      0.96       489
           4       1.00      0.28      0.44        46

    accuracy                           0.92      1125
   macro avg       0.94      0.69      0.75      1125
weighted avg       0.92      0.92      0.91      1125



In [18]:
pip install xgboost


Note: you may need to restart the kernel to use updated packages.


In [23]:
# Logistic Regression
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_test)
accuracy_logreg = accuracy_score(y_test, y_pred_logreg)
report_logreg = classification_report(y_test, y_pred_logreg)

# Support Vector Machine
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
report_svm = classification_report(y_test, y_pred_svm)





# Displaying results
print("Logistic Regression Accuracy:", accuracy_logreg)
print("Logistic Regression Classification Report:\n", report_logreg)

print("\nSupport Vector Machine Accuracy:", accuracy_svm)
print("Support Vector Machine Classification Report:\n", report_svm)





c:\Users\Nihan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nihan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nihan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Logistic Regression Accuracy: 0.5422222222222223
Logistic Regression Classification Report:
               precision    recall  f1-score   support

           1       0.55      0.59      0.57       499
           2       0.00      0.00      0.00        91
           3       0.53      0.64      0.58       489
           4       1.00      0.09      0.16        46

    accuracy                           0.54      1125
   macro avg       0.52      0.33      0.33      1125
weighted avg       0.52      0.54      0.51      1125


Support Vector Machine Accuracy: 0.544
Support Vector Machine Classification Report:
               precision    recall  f1-score   support

           1       0.54      0.63      0.58       499
           2       0.00      0.00      0.00        91
           3       0.55      0.61      0.58       489
           4       0.00      0.00      0.00        46

    accuracy                           0.54      1125
   macro avg       0.27      0.31      0.29      1125
weigh

c:\Users\Nihan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nihan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nihan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Transform the text data into TF-IDF vectors
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Initialize and train the Naive Bayes classifier
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)

# Make predictions on the test data
y_pred = naive_bayes.predict(X_test)


# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print("\nNavie bayes Machine Accuracy:", accuracy)
print("classification report:\n", report)


from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Create a SVM Classifier
xgb_classifier = XGBClassifier()
rf_classifier = RandomForestClassifier()

# Create the ensemble model
ensemble = VotingClassifier(estimators=[('xgb', xgb_classifier), ('rf', rf_classifier)], voting='hard')

# Train the model using the training sets
ensemble.fit(X_train, y_train)

# Predict the response for test dataset
y_pred_ensemble = ensemble.predict(X_test)

# Model Accuracy
print(f'Accuracy: {accuracy_score(y_test, y_pred_ensemble):.2f}')

# Classification Report
print('Classification Report for ensemble:\n', classification_report(y_test, y_pred_ensemble))





c:\Users\Nihan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nihan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nihan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,


Navie bayes Machine Accuracy: 0.501195219123506
classification report:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00       120
           2       0.46      0.45      0.46       455
           3       0.00      0.00      0.00        82
           4       0.52      0.71      0.60       598

    accuracy                           0.50      1255
   macro avg       0.25      0.29      0.26      1255
weighted avg       0.42      0.50      0.45      1255

Accuracy: 0.52
Classification Report for ensemble:
               precision    recall  f1-score   support

           1       0.15      0.02      0.03       120
           2       0.46      0.66      0.54       455
           3       0.00      0.00      0.00        82
           4       0.60      0.60      0.60       598

    accuracy                           0.52      1255
   macro avg       0.30      0.32      0.29      1255
weighted avg       0.47      0.52      0.48      1255



In [31]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
import lightgbm as lgb
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from collections import Counter

# Adjust the labels to start from 0
y_adjusted = y - 1

# Preprocess text data
sentences = [word_tokenize(text) for text in X]
word2vec_model = Word2Vec(sentences, min_count=1)

def text_to_avg_word2vec(text):
    words = word_tokenize(text)
    vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return [0] * word2vec_model.vector_size

X_word2vec = [text_to_avg_word2vec(text) for text in X]

# Apply SMOTE resampling
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_word2vec, y_adjusted)

# Display class distribution after resampling
print("Class distribution after resampling:", Counter(y_resampled))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Function to evaluate a model
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return accuracy, report

# Logistic Regression
logreg = LogisticRegression(max_iter=1000, random_state=42)
accuracy_logreg, report_logreg = evaluate_model(logreg, X_train, X_test, y_train, y_test)

# Support Vector Machine
svm = SVC(kernel='linear', random_state=42)
accuracy_svm, report_svm = evaluate_model(svm, X_train, X_test, y_train, y_test)

# XGBoost
xgb = XGBClassifier(random_state=42)
accuracy_xgb, report_xgb = evaluate_model(xgb, X_train, X_test, y_train, y_test)

# LightGBM
lgbm = lgb.LGBMClassifier(random_state=42)
accuracy_lgbm, report_lgbm = evaluate_model(lgbm, X_train, X_test, y_train, y_test)

# Naive Bayes (using TF-IDF since Naive Bayes doesn't work with negative values)
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(X)

X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, y_adjusted, test_size=0.2, random_state=42)

naive_bayes = MultinomialNB()
accuracy_nb, report_nb = evaluate_model(naive_bayes, X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf)

# Displaying results
print("Logistic Regression Accuracy:", accuracy_logreg)
print("Logistic Regression Classification Report:\n", report_logreg)

print("\nSupport Vector Machine Accuracy:", accuracy_svm)
print("Support Vector Machine Classification Report:\n", report_svm)

print("\nXGBoost Accuracy:", accuracy_xgb)
print("XGBoost Classification Report:\n", report_xgb)

print("\nLightGBM Accuracy:", accuracy_lgbm)
print("LightGBM Classification Report:\n", report_lgbm)

print("\nNaive Bayes Accuracy:", accuracy_nb)
print("Naive Bayes Classification Report:\n", report_nb)


Class distribution after resampling: Counter({3: 2846, 2: 2846, 0: 2846, 1: 2846})
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003418 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 9107, number of used features: 100
[LightGBM] [Info] Start training from score -1.387063
[LightGBM] [Info] Start training from score -1.391469
[LightGBM] [Info] Start training from score -1.382240
[LightGBM] [Info] Start training from score -1.384429
Logistic Regression Accuracy: 0.33992094861660077
Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       0.33      0.28      0.31       571
           1       0.34      0.20      0.26       581
           2       0.35      0.44      0.39       560
           3       0.33      0.44      0.38       565

    accuracy                           0.34  

c:\Users\Nihan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nihan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nihan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [21]:
pip install numpy nlpaug


   ---------------------------------------- 0.0/410.5 kB ? eta -:--:--
   --- ----------------------------------- 41.0/410.5 kB 991.0 kB/s eta 0:00:01
   --------------- ------------------------ 163.8/410.5 kB 1.6 MB/s eta 0:00:01
   ---------------------------- ----------- 297.0/410.5 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 410.5/410.5 kB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
import pandas as pd
import nlpaug.augmenter.word as naw
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler



# Extract features and labels
X = df['processed_review_content']
y = df['final_label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define augmentation technique
aug = naw.SynonymAug(aug_src='wordnet')

# Augment training data
# Augment training data
X_train_augmented = []
y_train_augmented = []

for text, label in zip(X_train, y_train):
    augmented_text = aug.augment(text)
    if isinstance(augmented_text, list) and augmented_text:  # Check if augmentation returns a non-empty list
        augmented_text = augmented_text[0]  # Use the first element of the list
        X_train_augmented.append(augmented_text.lower())  # Apply lower() to the augmented text
    else:
        X_train_augmented.append(text.lower())  # If augmentation fails, use original text
    y_train_augmented.append(label)



# Combine original and augmented data
X_train_combined = list(X_train) + X_train_augmented
y_train_combined = list(y_train) + y_train_augmented

# Vectorize the text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_combined)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Apply Random Over-Sampling (ROS)
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train_tfidf, y_train_combined)

# Train a Random Forest classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_resampled, y_train_resampled)

# Make predictions
y_pred = clf.predict(X_test_tfidf)


# Evaluate the model
print(classification_report(y_test, y_pred))


Accuracy: 0.53
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       120
           2       0.47      0.58      0.52       455
           3       0.00      0.00      0.00        82
           4       0.58      0.66      0.62       598

    accuracy                           0.53      1255
   macro avg       0.26      0.31      0.28      1255
weighted avg       0.45      0.53      0.48      1255



In [10]:
label_1_3_rows = df[df['final_label'].isin([1, 3])].copy()

In [13]:
label_1_3_rows.value_counts('final_label')

final_label
1    567
3    434
Name: count, dtype: int64

In [15]:
combined_df = pd.concat([df, label_1_3_rows], ignore_index=True)
combined_df.value_counts('final_label')

final_label
4    2846
2    2428
1    1134
3     868
Name: count, dtype: int64

In [17]:
shuffled_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)
shuffled_df.value_counts('final_label')


final_label
4    2846
2    2428
1    1134
3     868
Name: count, dtype: int64

In [33]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Encode the 'Emotion' column into numerical values
df['final_label_num'] = label_encoder.fit_transform(df['final_label'])

In [35]:
shuffled_df.columns

Index(['Unnamed: 0', 'review_content', 'student1_email', 'student1annotation',
       'student2_email', 'student2annotation', 'student3_email',
       'student3annotation', 'annotation_count', 'agreement_type',
       'final_label', 'processed_review_content'],
      dtype='object')

In [19]:
# from sklearn.model_selection import train_test_split

# # Assuming 'shuffled_df' is your DataFrame and 'label' is the target column
# X = shuffled_df['processed_review_content']
# y = shuffled_df['final_label']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [38]:
# vectorizer = TfidfVectorizer()

# # Transform the training and testing data
# X_train_tfidf = vectorizer.fit_transform(X_train)

# X_test_tfidf = vectorizer.transform(X_test)

In [18]:
# logreg = LogisticRegression()
# logreg.fit(X_train_tfidf, y_train)
# y_pred_logreg = logreg.predict(X_test_tfidf)

# # Evaluate the models

# print(f'Accuracy: {accuracy_score(y_test, y_pred_logreg):.2f}')
# print('Classification Report:\n', classification_report(y_test, y_pred_logreg))
# print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred_logreg))

In [17]:
# # Initialize the Naive Bayes classifier
# naive_bayes = MultinomialNB()

# # Train the classifier on the training data
# naive_bayes.fit(X_train_tfidf, y_train)

# # Make predictions on the test data
# y_pred = naive_bayes.predict(X_test_tfidf)

# # Evaluate the model
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy: {accuracy:.2f}')

# # Classification Report
# print('\nClassification Report:\n', classification_report(y_test, y_pred))

# # Confusion Matrix
# print('\nConfusion Matrix:\n', confusion_matrix(y_test, y_pred))

In [16]:
# rf_classifier = RandomForestClassifier()
# rf_classifier.fit(X_train_tfidf, y_train)
# y_pred_rf = rf_classifier.predict(X_test_tfidf)



# print(f'Accuracy: {accuracy_score(y_test, y_pred_rf):.2f}')
# print('Classification Report:\n', classification_report(y_test, y_pred_rf))
# print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred_rf))

In [14]:
# from sklearn import svm

# # Create a SVM Classifier
# svm_classifier = svm.SVC()

# # Train the model using the training sets
# svm_classifier.fit(X_train_tfidf, y_train)

# # Predict the response for test dataset
# y_pred_svm = svm_classifier.predict(X_test_tfidf)

# # Model Accuracy
# print(f'Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}')

# # Classification Report
# print('Classification Report:\n', classification_report(y_test, y_pred_svm))

# # Confusion Matrix
# print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred_svm))


In [73]:
# import xgboost as xgb

# # Create a XGBoost Classifier
# xgb_classifier = xgb.XGBClassifier()

# # Train the model using the training sets
# xgb_classifier.fit(X_train_tfidf, y_train)

# # Predict the response for test dataset
# y_pred_xgb = xgb_classifier.predict(X_test_tfidf)

# # Model Accuracy
# print(f'Accuracy: {accuracy_score(y_test, y_pred_xgb):.2f}')

# # Classification Report
# print('Classification Report:\n', classification_report(y_test, y_pred_xgb))

# # Confusion Matrix
# print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred_xgb))


In [15]:
# from sklearn.ensemble import VotingClassifier
# from xgboost import XGBClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# # Create a SVM Classifier
# xgb_classifier = XGBClassifier()
# rf_classifier = RandomForestClassifier()

# # Create the ensemble model
# ensemble = VotingClassifier(estimators=[('xgb', xgb_classifier), ('rf', rf_classifier)], voting='hard')

# # Train the model using the training sets
# ensemble.fit(X_train_tfidf, y_train)

# # Predict the response for test dataset
# y_pred_ensemble = ensemble.predict(X_test_tfidf)

# # Model Accuracy
# print(f'Accuracy: {accuracy_score(y_test, y_pred_ensemble):.2f}')

# # Classification Report
# print('Classification Report:\n', classification_report(y_test, y_pred_ensemble))

# # Confusion Matrix
# print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred_ensemble))



In [52]:
# pip install transformers torch